In [1]:
import tensorflow as tf
import numpy as np
import itertools
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib import pyplot

In [2]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score

In [3]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras import initializers, optimizers
from keras.layers import Input, BatchNormalization
from keras.layers import  Dense, Flatten, Activation, Dropout, Embedding
from keras.layers import LSTM, TimeDistributed, Permute,Reshape, Lambda, RepeatVector, merge, Input,Multiply
from keras.utils import np_utils
from keras.layers.wrappers import  Bidirectional
from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2 
from keras.models import load_model
import keras

Using TensorFlow backend.


In [4]:
from os import listdir
from os.path import isfile, join
import glob

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [6]:
mypath = '/home/chenming/ncrna/ncRDeep2/Data_Processing/RNAfold/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [7]:
my_files8 = glob.glob("/home/chenming/ncrna/ncRDeep/Data_Processing/Ten_Fold_Onehot_Data_h5/Onehot*.h5")
my_files8.sort()
my_files8[0:2]

['/home/chenming/ncrna/ncRDeep/Data_Processing/Ten_Fold_Onehot_Data_h5/Onehot_Fold_0_Test_Data_750.h5',
 '/home/chenming/ncrna/ncRDeep/Data_Processing/Ten_Fold_Onehot_Data_h5/Onehot_Fold_0_Train_Data_750.h5']

In [8]:
my_files7 = glob.glob("/home/chenming/ncrna/ncRDeep2/Data_Processing/RNAfold/*.h5")
my_files7.sort()
my_files7[0:2]

['/home/chenming/ncrna/ncRDeep2/Data_Processing/RNAfold/SS_test_0.h5',
 '/home/chenming/ncrna/ncRDeep2/Data_Processing/RNAfold/SS_test_1.h5']

In [9]:
my_files7[10]

'/home/chenming/ncrna/ncRDeep2/Data_Processing/RNAfold/SS_train_0.h5'

In [10]:
def get_file8(fold_no): #get train and test data from file by fold number
    hf_Train = h5.File(my_files8[fold_no*2+1],'r')
    hf_Test = h5.File(my_files8[fold_no*2],'r')
    X_train = hf_Train['Train_Data'] # Get train set
    X_train = np.array(X_train)
    Y_train = hf_Train['Label']      # Get train label
    Y_train = np.array(Y_train)
    X_test = hf_Test['Test_Data']     # Get test set
    X_test = np.array(X_test)
    Y_test = hf_Test['Label']       # Get test label
    Y_test = np.array(Y_test)
    Y_train = np_utils.to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = np_utils.to_categorical(Y_test, 13)    #  Process the label of test
    return X_train, Y_train, X_test, Y_test

In [11]:
def get_file7(fold_no): #get train and test data from file by fold number
    hf_Train = h5.File(my_files7[10+fold_no],'r')
    hf_Test = h5.File(my_files7[fold_no],'r')
    X_train = hf_Train['Train_Data'] # Get train set
    X_train = np.array(X_train)
    Y_train = hf_Train['Label']      # Get train label
    Y_train = np.array(Y_train)
    X_test = hf_Test['Test_Data']     # Get test set
    X_test = np.array(X_test)
    Y_test = hf_Test['Label']       # Get test label
    Y_test = np.array(Y_test)
    Y_train = np_utils.to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = np_utils.to_categorical(Y_test, 13)    #  Process the label of test
    return X_train, Y_train, X_test, Y_test

In [12]:
#class_names = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]
class_names = ['5S_rRNA', '5.8S_rRNA', 'tRNA', 'ribozymes', 'CD-box', 'miRNA', 'Intron_gpI', 'Intron_gpII', 'HACA-box', 'riboswitch', 'IRES', 'leader', 'scaRNA']
class_names

['5S_rRNA',
 '5.8S_rRNA',
 'tRNA',
 'ribozymes',
 'CD-box',
 'miRNA',
 'Intron_gpI',
 'Intron_gpII',
 'HACA-box',
 'riboswitch',
 'IRES',
 'leader',
 'scaRNA']

In [13]:
def plot_confusion_matrix(cm, classes,
                            normalize=False,
                            title='Confusion matrix',
                            cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    FONT_SIZE = 10
    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix\n============================")
    else:
        #cm = np.asfarray(cm,float64)
        print('Confusion matrix, without normalization\n============================')
    #print(cm)
    plt.figure(figsize=(5*2, 4*2))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=FONT_SIZE)
    plt.yticks(tick_marks, classes, fontsize=FONT_SIZE)
    fmt = '.3f' if normalize else '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                    horizontalalignment="center",
                    fontsize=FONT_SIZE,
                    color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label', fontsize=FONT_SIZE)
    plt.xlabel('Predicted label', fontsize=FONT_SIZE)
    plt.savefig('Conf_mat_avg.pdf')
    plt.show()

In [14]:
def Conv1Dme(f, k, xo):
    x1o=Conv1D(filters=f,kernel_size=k,strides=1,padding="same",kernel_initializer=initializers.random_uniform()) (xo)
    x1o = BatchNormalization()(x1o)
    x1o=Dropout(0.2)(x1o)
    x1o=Activation('relu')(x1o)
    #x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    return x1o

In [15]:
def FC1Dme(k, xo):
    xf=Dense(k,)(xo)
    xf = BatchNormalization()(xf)
    xf=Dropout(0.2)(xf)
    xf=Activation('relu')(xf)
    return xf

In [16]:
def dense_block_1(xin, f, k):
    f1 = f
    k1 = k

    x1 = Conv1Dme(f1, k1, xin)
    x11 = Conv1Dme(f1, k1, x1)
    x11 = Concatenate(axis=-1)([x1,x11])
    x11 = Conv1Dme(f1, k1, x11)
    x11 = Concatenate(axis=-1)([x1,x11])
    x11 = Conv1Dme(f1, k1, x11)
    x1=MaxPooling1D(pool_size=2, strides=2)(x11)
    
    return x1

In [17]:
X_train, Y_train, X_test, Y_test = get_file8(0)

In [18]:
X_train1, Y_train1, X_test1, Y_test1 = get_file7(0)

In [19]:
np.array_equal(X_train[0], X_train1[0,:,0:4])

True

In [20]:
print(X_train1.shape,X_test1.shape)

((5688, 750, 7), (632, 750, 7))


In [60]:
def model_dense2(): # Model
    inputs1 = Input(shape=(750, 4))
    inputs2 = Input(shape=(750, 3))
    
    x1=keras.layers.concatenate([inputs1,inputs2],axis=-1)
    
    x1 = Conv1Dme(512, 20, x1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = Conv1Dme(128, 6, x1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
            
    xf=Flatten()(x1)

    xf=FC1Dme(256,xf)
    xf=FC1Dme(64,xf)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=[inputs1,inputs2], outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    return model

In [61]:
model = model_dense2()
#print(model.summary())

In [62]:
es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit([X_train1[:,:,0:4],X_train1[:,:,4:7]], Y_train, validation_data=([X_test1[:,:,0:4],X_test1[:,:,4:7]], Y_test), epochs=500, verbose=1, callbacks=[es])
y = model.predict([X_test1[:,:,0:4],X_test1[:,:,4:7]])
y_test_non_category = [ np.argmax(t) for t in Y_test ]
y_predict_non_category = [ np.argmax(t) for t in y ]
auc = accuracy_score(y_test_non_category, y_predict_non_category)
print(auc)

Train on 5688 samples, validate on 632 samples
Epoch 1/500
5688/5688 [==============================] - 9s 2ms/step - loss: 1.6795 - acc: 0.4789 - val_loss: 1.2692 - val_acc: 0.6472
Epoch 2/500
5688/5688 [==============================] - 6s 998us/step - loss: 1.1188 - acc: 0.6791 - val_loss: 1.0895 - val_acc: 0.7294
Epoch 3/500
5688/5688 [==============================] - 6s 1ms/step - loss: 0.8792 - acc: 0.7664 - val_loss: 0.9722 - val_acc: 0.7326
Epoch 4/500
5688/5688 [==============================] - 6s 999us/step - loss: 0.7063 - acc: 0.8268 - val_loss: 0.8546 - val_acc: 0.8070
Epoch 5/500
5688/5688 [==============================] - 6s 1ms/step - loss: 0.5864 - acc: 0.8652 - val_loss: 0.7720 - val_acc: 0.8022
Epoch 6/500
5688/5688 [==============================] - 6s 1ms/step - loss: 0.4954 - acc: 0.8936 - val_loss: 0.7385 - val_acc: 0.8133
Epoch 7/500
5688/5688 [==============================] - 6s 1ms/step - loss: 0.4103 - acc: 0.9235 - val_loss: 0.7049 - val_acc: 0.8149
Epoc

In [69]:
def model_dense2(): # Model
    inputs1 = Input(shape=(750, 4))
    inputs2 = Input(shape=(750, 3))
    
    
    
    ###############################################################
    #          1st dense block
    
    x1 = Conv1Dme(512, 20, inputs1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = Conv1Dme(128, 6, x1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x2 = Conv1Dme(512, 20, inputs2)
    x2=MaxPooling1D(pool_size=2, strides=2)(x2)
    
    x2 = Conv1Dme(128, 6, x2)
    x2=MaxPooling1D(pool_size=2, strides=2)(x2)
    
    xf=keras.layers.concatenate([x1,x2],axis=-1)
    
    xf = Conv1Dme(128, 6, xf)
    xf=MaxPooling1D(pool_size=2, strides=2)(xf)
    
    #x1 = dense_block_1(x1, 128, 4)
    #x1 = dense_block_1(x1, 256, 5)
    #x1 = dense_block_1(x1, 256, 5)
            
    xf=Flatten()(xf)
    #xf2=Flatten()(x2)
    
    

    xf=FC1Dme(256,xf)
    xf=FC1Dme(64,xf)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=[inputs1,inputs2], outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    return model

In [70]:
model = model_dense2()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 750, 4)       0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 750, 3)       0                                            
__________________________________________________________________________________________________
conv1d_29 (Conv1D)              (None, 750, 512)     41472       input_18[0][0]                   
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, 750, 512)     31232       input_19[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [74]:
model = model_dense2()
es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit([X_train1[:,:,0:4],X_train1[:,:,4:7]], Y_train, validation_data=([X_test1[:,:,0:4],X_test1[:,:,4:7]], Y_test), epochs=500, verbose=1, callbacks=[es])
y = model.predict([X_test1[:,:,0:4],X_test1[:,:,4:7]])
y_test_non_category = [ np.argmax(t) for t in Y_test ]
y_predict_non_category = [ np.argmax(t) for t in y ]
auc = accuracy_score(y_test_non_category, y_predict_non_category)
print(auc)

Train on 5688 samples, validate on 632 samples
Epoch 1/500
5688/5688 [==============================] - 16s 3ms/step - loss: 1.8500 - acc: 0.4156 - val_loss: 1.5485 - val_acc: 0.5949
Epoch 2/500
5688/5688 [==============================] - 9s 2ms/step - loss: 1.3169 - acc: 0.6199 - val_loss: 1.3417 - val_acc: 0.6503
Epoch 3/500
5688/5688 [==============================] - 9s 2ms/step - loss: 1.0822 - acc: 0.6930 - val_loss: 1.1181 - val_acc: 0.7231
Epoch 4/500
5688/5688 [==============================] - 9s 2ms/step - loss: 0.9171 - acc: 0.7514 - val_loss: 1.0664 - val_acc: 0.7453
Epoch 5/500
5688/5688 [==============================] - 9s 2ms/step - loss: 0.8055 - acc: 0.7827 - val_loss: 0.9450 - val_acc: 0.7785
Epoch 6/500
5688/5688 [==============================] - 9s 2ms/step - loss: 0.7162 - acc: 0.8140 - val_loss: 0.8981 - val_acc: 0.7848
Epoch 7/500
5688/5688 [==============================] - 9s 2ms/step - loss: 0.6194 - acc: 0.8397 - val_loss: 0.7604 - val_acc: 0.8165
Epoch 8

In [56]:
def model_dense2(): # Model
    inputs1 = Input(shape=(750, 4))
    #inputs2 = Input(shape=(750, 3))
    
    #x1=keras.layers.concatenate([inputs1,inputs2],axis=-1)
    
    ###############################################################
    #          1st dense block
    
    x1 = Conv1Dme(512, 20, inputs1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = Conv1Dme(128, 6, x1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    
    #x1 = dense_block_1(x1, 128, 4)
    #x1 = dense_block_1(x1, 256, 5)
    #x1 = dense_block_1(x1, 256, 5)
            
    xf=Flatten()(x1)

    xf=FC1Dme(256,xf)
    xf=FC1Dme(64,xf)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=inputs1, outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    return model

In [57]:
model = model_dense2()
#print(model.summary())
es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit(X_train1[:,:,0:4], Y_train, validation_data=(X_test1[:,:,0:4], Y_test), epochs=500, verbose=1, callbacks=[es])
y = model.predict(X_test1[:,:,0:4])
y_test_non_category = [ np.argmax(t) for t in Y_test ]
y_predict_non_category = [ np.argmax(t) for t in y ]
auc = accuracy_score(y_test_non_category, y_predict_non_category)
print(auc)

Train on 5688 samples, validate on 632 samples
Epoch 1/500
5688/5688 [==============================] - 8s 1ms/step - loss: 1.7604 - acc: 0.4436 - val_loss: 1.3278 - val_acc: 0.6123
Epoch 2/500
5688/5688 [==============================] - 6s 976us/step - loss: 1.1767 - acc: 0.6660 - val_loss: 1.1626 - val_acc: 0.6741
Epoch 3/500
5688/5688 [==============================] - 6s 971us/step - loss: 0.9457 - acc: 0.7423 - val_loss: 0.9554 - val_acc: 0.7421
Epoch 4/500
5688/5688 [==============================] - 6s 982us/step - loss: 0.7615 - acc: 0.8129 - val_loss: 0.8281 - val_acc: 0.7769
Epoch 5/500
5688/5688 [==============================] - 6s 980us/step - loss: 0.6310 - acc: 0.8565 - val_loss: 0.7767 - val_acc: 0.7991
Epoch 6/500
5688/5688 [==============================] - 6s 976us/step - loss: 0.5230 - acc: 0.8845 - val_loss: 0.7963 - val_acc: 0.8085
Epoch 7/500
5688/5688 [==============================] - 6s 982us/step - loss: 0.4410 - acc: 0.9102 - val_loss: 0.6627 - val_acc: 0.8

In [94]:
vec_train = []
for j in range(len(X_train1)):
    newvecs = []
    for i in range(len(X_train1[0,:,:])):
        newvec1 = np.multiply(X_train1[j,i,0:4],X_train1[j,i,4])
        newvec2 = np.multiply(X_train1[j,i,0:4],X_train1[j,i,5])
        newvec3 = np.multiply(X_train1[j,i,0:4],X_train1[j,i,6])
        newvec = np.concatenate((newvec1,newvec2,newvec3),axis=0)
        newvecs.append(newvec)
    newvecs = np.array(newvecs)
    vec_train.append(newvecs)
vec_train = np.array(vec_train)
        #print(newvec)

In [96]:
vec_test = []
for j in range(len(X_test1)):
    newvecs = []
    for i in range(len(X_test1[0,:,:])):
        newvec1 = np.multiply(X_test1[j,i,0:4],X_test1[j,i,4])
        newvec2 = np.multiply(X_test1[j,i,0:4],X_test1[j,i,5])
        newvec3 = np.multiply(X_test1[j,i,0:4],X_test1[j,i,6])
        newvec = np.concatenate((newvec1,newvec2,newvec3),axis=0)
        newvecs.append(newvec)
    newvecs = np.array(newvecs)
    vec_test.append(newvecs)
vec_test = np.array(vec_test)

In [82]:
print(X_train1[0,0,0:4],X_train1[0,0,4:7])

(array([1, 0, 0, 0]), array([1, 0, 0]))


In [90]:
newvecs = np.array(newvecs)
newvecs

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [97]:
vec_test.shape

(632, 750, 12)

In [106]:
def model_vec1(): # Model
    inputs1 = Input(shape=(750, 12))
    
    x1 = Conv1Dme(512, 20, inputs1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = Conv1Dme(256, 8, x1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = Conv1Dme(128, 3, x1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
            
    xf=Flatten()(x1)

    xf=FC1Dme(256,xf)
    xf=FC1Dme(64,xf)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=inputs1, outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    return model

In [107]:
model = model_vec1()
#print(model.summary())
es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit(vec_train, Y_train, validation_data=(vec_test, Y_test), epochs=500, verbose=1, callbacks=[es])
y = model.predict(vec_test)
y_test_non_category = [ np.argmax(t) for t in Y_test ]
y_predict_non_category = [ np.argmax(t) for t in y ]
auc = accuracy_score(y_test_non_category, y_predict_non_category)
print(auc)

Train on 5688 samples, validate on 632 samples
Epoch 1/500
5688/5688 [==============================] - 14s 2ms/step - loss: 1.8994 - acc: 0.3877 - val_loss: 1.5780 - val_acc: 0.5380
Epoch 2/500
5688/5688 [==============================] - 7s 1ms/step - loss: 1.3782 - acc: 0.5803 - val_loss: 1.3518 - val_acc: 0.6361
Epoch 3/500
5688/5688 [==============================] - 7s 1ms/step - loss: 1.1451 - acc: 0.6637 - val_loss: 1.2322 - val_acc: 0.6756
Epoch 4/500
5688/5688 [==============================] - 7s 1ms/step - loss: 0.9814 - acc: 0.7222 - val_loss: 1.1736 - val_acc: 0.6693
Epoch 5/500
5688/5688 [==============================] - 7s 1ms/step - loss: 0.8613 - acc: 0.7588 - val_loss: 1.0763 - val_acc: 0.6835
Epoch 6/500
5688/5688 [==============================] - 7s 1ms/step - loss: 0.7652 - acc: 0.7952 - val_loss: 1.0019 - val_acc: 0.7231
Epoch 7/500
5688/5688 [==============================] - 7s 1ms/step - loss: 0.6768 - acc: 0.8194 - val_loss: 0.9352 - val_acc: 0.7310
Epoch 8

In [123]:
def model_dense2(): # Model
    inputs1 = Input(shape=(750, 4))
    inputs2 = Input(shape=(750, 3))
    
    ###############################################################
    #          1st dense block
    
    x1 = Conv1Dme(64, 3, inputs1)
    x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = dense_block_1(x1, 128, 3)
    x1 = dense_block_1(x1, 256, 4)
    x1 = dense_block_1(x1, 256, 5)
    
    
    ###############################################################
    #          2nd dense block
    
    x2 = Conv1Dme(64, 3, inputs2)
    x2=MaxPooling1D(pool_size=2, strides=2)(x2)
    
    x2 = dense_block_1(x2, 128, 3)
    x2 = dense_block_1(x2, 256, 4)
    x2 = dense_block_1(x2, 256, 5)
    #x2 = dense_block_1(x2, 128, 4)
    #x2 = dense_block_1(x2, 256, 5)
    #x2 = dense_block_1(x2, 256, 5)
            
    xf1=Flatten()(x1)
    xf2=Flatten()(x2)
    
    xf=keras.layers.concatenate([xf1,xf2],axis=-1)

    xf=FC1Dme(256,xf)
    xf=FC1Dme(64,xf)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=[inputs1,inputs2], outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    return model

In [124]:
model = model_dense2()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_53 (InputLayer)           (None, 750, 4)       0                                            
__________________________________________________________________________________________________
input_54 (InputLayer)           (None, 750, 3)       0                                            
__________________________________________________________________________________________________
conv1d_328 (Conv1D)             (None, 750, 64)      832         input_53[0][0]                   
__________________________________________________________________________________________________
conv1d_341 (Conv1D)             (None, 750, 64)      640         input_54[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [125]:
model = model_dense2()
es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit([X_train1[:,:,0:4],X_train1[:,:,4:7]], Y_train, validation_data=([X_test1[:,:,0:4],X_test1[:,:,4:7]], Y_test), epochs=500, verbose=1, callbacks=[es])
y = model.predict([X_test1[:,:,0:4],X_test1[:,:,4:7]])
y_test_non_category = [ np.argmax(t) for t in Y_test ]
y_predict_non_category = [ np.argmax(t) for t in y ]
auc = accuracy_score(y_test_non_category, y_predict_non_category)
print(auc)

Train on 5688 samples, validate on 632 samples
Epoch 1/500
5688/5688 [==============================] - 52s 9ms/step - loss: 2.1597 - acc: 0.2737 - val_loss: 2.8924 - val_acc: 0.0791
Epoch 2/500
5688/5688 [==============================] - 21s 4ms/step - loss: 1.7240 - acc: 0.4353 - val_loss: 2.5925 - val_acc: 0.2152
Epoch 3/500
5688/5688 [==============================] - 20s 4ms/step - loss: 1.4601 - acc: 0.5346 - val_loss: 2.5099 - val_acc: 0.1693
Epoch 4/500
5688/5688 [==============================] - 21s 4ms/step - loss: 1.2478 - acc: 0.6171 - val_loss: 2.1430 - val_acc: 0.2547
Epoch 5/500
5688/5688 [==============================] - 21s 4ms/step - loss: 1.1028 - acc: 0.6709 - val_loss: 2.1855 - val_acc: 0.2500
Epoch 6/500
5688/5688 [==============================] - 21s 4ms/step - loss: 1.0090 - acc: 0.7045 - val_loss: 2.0752 - val_acc: 0.2832
Epoch 7/500
5688/5688 [==============================] - 21s 4ms/step - loss: 0.8972 - acc: 0.7407 - val_loss: 1.7131 - val_acc: 0.4304
E

In [132]:
def model_dense3(): # Model
    inputs1 = Input(shape=(750, 4))
    inputs2 = Input(shape=(750, 3))
    
    ###############################################################
    #          1st dense block
    
    x1 = Conv1Dme(64, 3, inputs1)
    x1 = MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = dense_block_1(x1, 128, 3)
    x1 = dense_block_1(x1, 256, 4)
    x1 = dense_block_1(x1, 256, 5)
    
    
    ###############################################################
    #          2nd dense block
    
    x2 = Conv1Dme(64, 3, inputs2)
    x2 = MaxPooling1D(pool_size=2, strides=2)(x2)
    
    x2 = dense_block_1(x2, 128, 3)
    x2 = dense_block_1(x2, 256, 4)
    x2 = dense_block_1(x2, 256, 5)
    
    xf=keras.layers.concatenate([x1,x2],axis=-1)
    
    xf = dense_block_1(xf, 256, 3)
    xf = Flatten()(xf)

    xf=FC1Dme(256,xf)
    xf=FC1Dme(64,xf)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=[inputs1,inputs2], outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    return model

In [133]:
model = model_dense3()
es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit([X_train1[:,:,0:4],X_train1[:,:,4:7]], Y_train, validation_data=([X_test1[:,:,0:4],X_test1[:,:,4:7]], Y_test), epochs=500, verbose=1, callbacks=[es])
y = model.predict([X_test1[:,:,0:4],X_test1[:,:,4:7]])
y_test_non_category = [ np.argmax(t) for t in Y_test ]
y_predict_non_category = [ np.argmax(t) for t in y ]
auc = accuracy_score(y_test_non_category, y_predict_non_category)
print(auc)

Train on 5688 samples, validate on 632 samples
Epoch 1/500
5688/5688 [==============================] - 67s 12ms/step - loss: 2.3389 - acc: 0.2189 - val_loss: 3.0489 - val_acc: 0.0791
Epoch 2/500
5688/5688 [==============================] - 23s 4ms/step - loss: 2.0380 - acc: 0.3066 - val_loss: 3.1516 - val_acc: 0.0918
Epoch 3/500
5688/5688 [==============================] - 23s 4ms/step - loss: 1.8973 - acc: 0.3699 - val_loss: 3.3362 - val_acc: 0.0791
Epoch 4/500
5688/5688 [==============================] - 23s 4ms/step - loss: 1.7483 - acc: 0.4311 - val_loss: 3.6514 - val_acc: 0.0791
Epoch 5/500
5688/5688 [==============================] - 23s 4ms/step - loss: 1.5719 - acc: 0.5058 - val_loss: 3.7309 - val_acc: 0.0791
Epoch 6/500
5688/5688 [==============================] - 23s 4ms/step - loss: 1.4503 - acc: 0.5452 - val_loss: 3.7068 - val_acc: 0.0791
Epoch 7/500
5688/5688 [==============================] - 23s 4ms/step - loss: 1.3530 - acc: 0.5788 - val_loss: 3.6936 - val_acc: 0.0791


In [25]:
def dense_block_4(xin, f, k):
    f1 = f
    k1 = k

    x1 = Conv1Dme(f1, k1, xin)
    x11 = Conv1Dme(f1, k1, x1)
    x11 = Concatenate(axis=-1)([x1,x11])
    x11 = Conv1Dme(f1, k1, x11)
    x11 = Concatenate(axis=-1)([x1,x11])
    x11 = Conv1Dme(f1, k1, x11)
    x1=MaxPooling1D(pool_size=2, strides=2)(x11)
    
    return x1

In [149]:
def model_dense4(): # Model
    inputs1 = Input(shape=(750, 4))
    inputs2 = Input(shape=(750, 3))
    
    ###############################################################
    #          1st dense block
    
    x1 = Conv1Dme(64, 3, inputs1)
    x1 = MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = dense_block_4(x1, 128, 4)
    x1 = dense_block_4(x1, 256, 5)
    x1 = dense_block_4(x1, 256, 5)
    
    
    ###############################################################
    #          2nd dense block
    
    x2 = Conv1Dme(64, 3, inputs2)
    x2 = MaxPooling1D(pool_size=2, strides=2)(x2)
    
    x2 = dense_block_4(x2, 128, 4)
    x2 = dense_block_4(x2, 256, 5)
    x2 = dense_block_4(x2, 256, 5)
    
    xf=keras.layers.concatenate([x1,x2],axis=-1)
    
    xf = dense_block_4(xf, 256, 4)
    xf = Flatten()(xf)

    xf=FC1Dme(256,xf)
    xf=FC1Dme(64,xf)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=[inputs1,inputs2], outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    return model

In [150]:
auc_mat_750c1 = []
history_750c1 = []
conf_mat_750c1 = []
history_750c1 = {}
class_report_750c1 = {}
for i in range(10):
    X_train, Y_train, X_test, Y_test = get_file7(i)
    model = model_dense4()
    es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    history = model.fit([X_train[:,:,0:4],X_train[:,:,4:7]], Y_train, validation_data=([X_test[:,:,0:4],X_test[:,:,4:7]], Y_test), epochs=500, verbose=0, callbacks=[es])
    model.save_weights("Checkpoints/Dense_RNAfold_1002_%i_fold.h5" %i)
    y = model.predict([X_test[:,:,0:4],X_test[:,:,4:7]])
    y_test_non_category = [ np.argmax(t) for t in Y_test ]
    y_predict_non_category = [ np.argmax(t) for t in y ]
    auc = accuracy_score(y_test_non_category, y_predict_non_category)
    precision,recall,fscore,support=score(y_test_non_category, y_predict_non_category,average='macro')
    mcc = matthews_corrcoef(y_test_non_category, y_predict_non_category)
    conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)
    classification_reports = classification_report(y_test_non_category, y_predict_non_category)
    print(auc,recall,precision,fscore,mcc)
    auc_mat_750c1.append([[auc],[recall],[precision],[fscore],[mcc]])
    #history_5.append([history.history])
    history_750c1['fold%i'%i]=history.history
    conf_mat_750c1.append([conf_mat])
    class_report_750c1['fold%i'%i]=classification_reports

(0.9208860759493671, 0.9178846153846154, 0.924348554678574, 0.9191159287707931, 0.9145298200274602)
(0.9272151898734177, 0.9188461538461538, 0.9291180206490424, 0.9212421825991927, 0.9213252215304497)
(0.9224683544303798, 0.9194230769230768, 0.9263536455696335, 0.9215026593774243, 0.9162229745094559)
(0.9113924050632911, 0.9060576923076924, 0.911446838380263, 0.906767810877328, 0.9042386424052794)
(0.9050632911392406, 0.9042307692307693, 0.905660910995597, 0.9020540255302592, 0.897551618998997)
(0.9367088607594937, 0.9315384615384615, 0.9369622273630032, 0.9331182012500321, 0.9314853590133775)
(0.9193037974683544, 0.9111538461538462, 0.9202301327190539, 0.9129073356763678, 0.9127822587563296)
(0.9193037974683544, 0.9154807692307692, 0.9225622872481003, 0.9168512725131162, 0.9127800713795204)
(0.9224683544303798, 0.9185576923076924, 0.9267359646935243, 0.9208986397408669, 0.9162976647732292)
(0.9161392405063291, 0.9054807692307691, 0.919673735878183, 0.907204683919741, 0.909588260242888

In [147]:
X_train, Y_train, X_test, Y_test = get_file7(i)

In [26]:
def model_dense5(): # Model
    inputs1 = Input(shape=(750, 4))
    inputs2 = Input(shape=(750, 3))
    
    ###############################################################
    #          1st dense block
    
    x1 = Conv1Dme(64, 3, inputs1)
    x1 = MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = dense_block_4(x1, 128, 4)
    x1 = dense_block_4(x1, 256, 5)
    #x1 = dense_block_4(x1, 256, 5)
    
    
    ###############################################################
    #          2nd dense block
    
    x2 = Conv1Dme(64, 3, inputs2)
    x2 = MaxPooling1D(pool_size=2, strides=2)(x2)
    
    x2 = dense_block_4(x2, 128, 4)
    x2 = dense_block_4(x2, 256, 5)
    #x2 = dense_block_4(x2, 256, 5)
    
    xf=keras.layers.concatenate([x1,x2],axis=-1)
    
    xf = dense_block_4(xf, 256, 4)
    xf = Flatten()(xf)

    xf=FC1Dme(256,xf)
    xf=FC1Dme(64,xf)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=[inputs1,inputs2], outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    return model

In [27]:
auc_mat_750c1 = []
history_750c1 = []
conf_mat_750c1 = []
history_750c1 = {}
class_report_750c1 = {}
for i in range(10):
    X_train, Y_train, X_test, Y_test = get_file7(i)
    model = model_dense5()
    es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    history = model.fit([X_train[:,:,0:4],X_train[:,:,4:7]], Y_train, validation_data=([X_test[:,:,0:4],X_test[:,:,4:7]], Y_test), epochs=500, verbose=0, callbacks=[es])
    model.save_weights("Checkpoints/Dense_RNAfold_1003_%i_fold.h5" %i)
    y = model.predict([X_val[:,:,0:4],X_val[:,:,4:7]])
    y_test_non_category = [ np.argmax(t) for t in Y_val ]
    y_predict_non_category = [ np.argmax(t) for t in y ]
    auc = accuracy_score(y_test_non_category, y_predict_non_category)
    precision,recall,fscore,support=score(y_test_non_category, y_predict_non_category,average='macro')
    mcc = matthews_corrcoef(y_test_non_category, y_predict_non_category)
    conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)
    classification_reports = classification_report(y_test_non_category, y_predict_non_category)
    print(auc,recall,precision,fscore,mcc)
    auc_mat_750c1.append([[auc],[recall],[precision],[fscore],[mcc]])
    #history_5.append([history.history])
    history_750c1['fold%i'%i]=history.history
    conf_mat_750c1.append([conf_mat])
    class_report_750c1['fold%i'%i]=classification_reports

(0.9353846153846154, 0.9353846153846155, 0.9376353492180876, 0.9355291637839754, 0.9301571263499605)
(0.9276923076923077, 0.9276923076923077, 0.9316479057578468, 0.9273583617107536, 0.922058033325592)
(0.9373076923076923, 0.9373076923076921, 0.9395116022421947, 0.9377707462533988, 0.9321929919716658)
(0.9338461538461539, 0.9338461538461538, 0.9362606000710788, 0.9339499644551006, 0.9285105707764049)
(0.925, 0.9250000000000002, 0.9292198910641495, 0.9255773913465568, 0.9190484446349905)
(0.9284615384615384, 0.9284615384615387, 0.9301348267778005, 0.9285386396260225, 0.9226333774782174)
(0.9088461538461539, 0.908846153846154, 0.9282430700593006, 0.9124683873037504, 0.902951583289637)
(0.9338461538461539, 0.9338461538461539, 0.9357461737909765, 0.9338247657839154, 0.9284880942468589)
(0.9307692307692308, 0.9307692307692308, 0.9336310101226287, 0.9305855194159665, 0.9252755477547618)
(0.9265384615384615, 0.9265384615384615, 0.9323303053768869, 0.9274475655613176, 0.9208493018396655)


In [28]:
auc_mat_750c1 = np.array(auc_mat_750c1)
for i in range(5):    
    print(np.average(auc_mat_750c1[:,i]))

0.9287692307692307
0.9287692307692307
0.9334360734480951
0.9293050505240756
0.9232165071667755


In [169]:
X_val[:,:,0:4].shape

(2600, 750, 4)

In [22]:
hf_val = h5.File(my_files7[20],'r')
X_val = hf_val['Test_Data']     # Get test set
X_val = np.array(X_val)
Y_val = hf_val['Label']       # Get test label
Y_val = np.array(Y_val)
Y_val = np_utils.to_categorical(Y_val, 13)    #  Process the label of test

In [21]:
my_files7[20]

'/home/chenming/ncrna/ncRDeep2/Data_Processing/RNAfold/SS_vals.h5'